# Example: Processing Employee Attendance


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time, timedelta

## Python date time primer


In [ ]:
# Create datetime
dt = datetime(year=2024, month=2, day=1, hour=8, minute=52)
print(dt)

In [ ]:
# Parse datetime from string
dt = datetime.strptime("2024-02-01", "%Y-%m-%d")
print(dt)

In [ ]:
# Format string
dt = datetime(year=2024, month=2, day=1, hour=8, minute=52)
dt.strftime("%Y/%m/%d %H:%M")

In [ ]:
# Create date
dt_date = date(year=2024, month=2, day=1)
print(dt_date)

In [ ]:
# Create time
dt_time = time(hour=7, minute=0)
print(dt_time)

In [ ]:
# Time delta
td = timedelta(days=1)
print(td)
print(td.total_seconds())

In [ ]:
dt = datetime(2024, 2, 1) + timedelta(days=1)
print(dt)

In [ ]:
# Subtract time
# https://stackoverflow.com/a/49528739
enter = time(hour=1)  # Example enter time
exit = time(hour=2)  # Example start time
enter_delta = timedelta(hours=enter.hour, minutes=enter.minute, seconds=enter.second)
exit_delta = timedelta(hours=exit.hour, minutes=exit.minute, seconds=exit.second)
difference_delta = exit_delta - enter_delta
print(difference_delta.total_seconds() / 60)

## Attendance Analysis


In [ ]:
dfr = pd.read_excel("./data/clock_inout.xlsx")

In [ ]:
dfr.head()

In [ ]:
dfr = dfr[["ชื่อ-นามสกุล", "Date", 1, 2, 3]].rename(
    columns={"ชื่อ-นามสกุล": "name", "Date": "date", 1: "c1", 2: "c2", 3: "c3"}
)
display(dfr.head(3))

In [ ]:
dfr.duplicated().sum()

In [ ]:
print(dfr.shape)
filtNull = dfr[["c1", "c2", "c3"]].isnull().all(axis=1)
dfr = dfr[~filtNull]
dfr.shape

In [ ]:
def parseDate(dateStr):
    sp = dateStr.split("/")
    day = sp[0]
    month = sp[1]
    year = int(sp[2]) - 543
    return pd.to_datetime(f"{year}/{month}/{day}", format="%Y/%m/%d")


# Convert to datetime
dfr["date"] = dfr["date"].apply(parseDate)

# Convert to date
dfr["date"] = dfr["date"].dt.date

# Check
print(type(dfr["date"].iloc[0]))

In [ ]:
# Convert to time
dfr["c1"] = pd.to_datetime(dfr["c1"], format="%H:%M").dt.time
dfr["c2"] = pd.to_datetime(dfr["c2"], format="%H:%M").dt.time
dfr["c3"] = pd.to_datetime(dfr["c3"], format="%H:%M").dt.time


In [ ]:
dfr.head()

In [ ]:
# Remove rows with hours outside acceptable range
def checkTimeOutsideRange(sr):
    return (sr < time(hour=6)) | (sr > time(hour=22))


filt1 = checkTimeOutsideRange(dfr["c1"])
filt2 = checkTimeOutsideRange(dfr["c2"])
filt3 = checkTimeOutsideRange(dfr["c3"])

filtOutsideRange = filt1 | filt2 | filt3
dfr[filtOutsideRange]

In [ ]:
dfr = dfr[~filtOutsideRange]

In [ ]:
dfr["incompleteInOut"] = False
filtOneCheckIn = (~dfr[["c1", "c2", "c3"]].isnull()).sum(axis=1) == 1
dfr.loc[filtOneCheckIn, "incompleteInOut"] = True

In [ ]:
# Add time for incomplete check-in/out
def addTimeForIncompleteCheckInOut(sr):
    dt_time = sr["c1"]
    # Determine whether the missing is the morning in or evening out.
    if dt_time < time(hour=13):  # 1pm
        sr["c2"] = time(hour=18)  # 6pm
    else:
        sr["c2"] = time(hour=9)  # 9am
    return sr


filtIIO = dfr["incompleteInOut"]
dfr.loc[filtIIO, :] = dfr.loc[filtIIO, :].apply(addTimeForIncompleteCheckInOut, axis=1)

In [ ]:
dfr.loc[filtIIO].head()

In [ ]:
def calculateInOut(row):
    times = row.loc[["c1", "c2", "c3"]].dropna()
    res = times.agg(["min", "max"])
    return pd.concat([row, res])


dfr = dfr.apply(calculateInOut, axis=1)
dfr = dfr.rename(columns={"min": "in", "max": "out"})
dfr.head()

In [ ]:
dfr["isInLate"] = dfr["in"] > time(hour=9)
dfr["isOutEarly"] = dfr["out"] < time(hour=18)
dfr.head()

In [ ]:
# You cannot substract time and time. Need to convert to timedelta first.
def calInLateMin(dt_time):
    deltaIn = timedelta(hours=dt_time.hour, minutes=dt_time.minute)
    deltaStart = timedelta(hours=9)
    lateMin = (deltaIn - deltaStart).total_seconds() / 60
    if lateMin < 0:
        lateMin = 0
    return lateMin


dfr["inLateMin"] = dfr["in"].apply(calInLateMin)
dfr.head()

In [ ]:
def calOutEarlyMin(dt_time):
    deltaOut = timedelta(hours=dt_time.hour, minutes=dt_time.minute)
    deltaEnd = timedelta(hours=18)
    earlyMon = (deltaEnd - deltaOut).total_seconds() / 60
    if earlyMon < 0:
        earlyMon = 0
    return earlyMon


dfr["outEarlyMin"] = dfr["out"].apply(calOutEarlyMin)
dfr.head()

In [ ]:
def calWorkingDuration(row):
    timeIn = row["in"]
    timeOut = row["out"]
    deltaIn = timedelta(hours=timeIn.hour, minutes=timeIn.minute)
    deltaOut = timedelta(hours=timeOut.hour, minutes=timeOut.minute)
    return (deltaOut - deltaIn).total_seconds() / 60


dfr["workingDuration"] = dfr.apply(calWorkingDuration, axis=1)
dfr.head()

In [ ]:
dfr["overWorkMin"] = dfr["workingDuration"] - (9 * 60)
dfr.head()

In [ ]:
# Get working days
dtRangesMonth = pd.date_range(start="2024-02-01", end="2024-02-28")

dtRangesWorking = []
for dtr in dtRangesMonth:
    if dtr.weekday() <= 5:  # Monday to Saturday
        dtRangesWorking.append(dtr.date())

In [ ]:
def matchWorkingDate(dft):
    dfWorking = pd.DataFrame(data={"working_date": dtRangesWorking})
    dfm = pd.merge(dfWorking, dft, left_on="working_date", right_on="date", how="left")
    dfm["present"] = dfm["date"].notnull()
    dfm["absent"] = dfm["date"].isnull()
    return dfm


dfg = dfr.groupby(by="name")
dfgm = dfg.apply(matchWorkingDate, include_groups=False)


# Testing
# dfg = dfr.groupby(by="name")
# dft = dfg.get_group("รุ้ง")
# dfWorking = pd.DataFrame(data={"working_date": dtRangesWorking})
# dfm = pd.merge(dfWorking, dft, left_on="working_date", right_on="date", how="left")
# name = dfm["name"].value_counts().index[0]
# dfm["name"] = name
# dfm

In [ ]:
dfgm = dfgm.reset_index().drop(columns="level_1")

In [ ]:
dfgm.columns

In [ ]:
out1 = (
    dfgm.groupby(by=["name"])
    .agg(
        {
            "present": "sum",
            "absent": "sum",
            "workingDuration": lambda s: s.mean() / 60,
            "overWorkMin": "sum",
            "incompleteInOut": "sum",
            "inLateMin": "sum",
            "outEarlyMin": "sum",
        }
    )
    .rename(
        columns={
            "workingDuration": "workingDuration (mean)",
        }
    )
)
display(out1)

In [ ]:
out2 = dfgm.pivot(index="name", columns="working_date", values="overWorkMin")
display(out2)

In [ ]:
# Using pivot_table, not pivot
out3 = dfgm.pivot_table(
    index="name", columns="working_date", values="present", aggfunc="sum"
)
display(out3)

In [ ]:
# Using pivot_table, not pivot
out4 = dfgm.pivot_table(
    index="name",
    columns="working_date",
    values="workingDuration",
    aggfunc=lambda s: np.round(s.mean() / 60, 2),
)
display(out4)

In [ ]:
# Write dataframes to Excel with multiple sheets
names = ["summary", "overWorkMin", "present", "workingDuration (hour)"]
dataframes = [out1, out2, out3, out4]
with pd.ExcelWriter("out_attendance.xlsx") as writer:
    for name, frame in zip(names, dataframes):
        frame.to_excel(writer, sheet_name=name, index=True)